In [1]:
import tensorflow as tf 
if tf.test.gpu_device_name(): 
    print('Default GPU Device:{}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

Default GPU Device:/device:GPU:0


In [2]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import pandas as pd
import sklearn as sk
from matplotlib import pyplot as plt 
from transformers import AutoTokenizer, AutoModel
from os import listdir
from os.path import isfile
import os
import pickle
import sys
import os.path
from os import path
from transformers import TFAutoModel
from transformers import BertTokenizer

import tensorflow as tf
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline

codebert_tokenizer = AutoTokenizer.from_pretrained("microsoft/codebert-base")
bert_tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

#!pip install transformers==4.20.0

SEED = 42

tf.random.set_seed(SEED)

use_function_metrics = True

In [3]:
# train = pd.read_excel(folder + 'Train_old.xlsx')
# test = pd.read_excel(folder + 'Test_old.xlsx')
train = pd.read_excel('Train_new.xlsx')
test = pd.read_excel('Test_new.xlsx')
train = train[train["message"].notna()]
test = test[test["message"].notna()]

train_comments = train['message']
test_comments = test['message']

train_y = train['comment_group'].astype('str')
test_y = test['comment_group'].astype('str')
len(train),len(test),len(train_comments),len(test_comments),len(train_y),len(test_y)

(1463, 365, 1463, 365, 1463, 365)

In [4]:
X_df_all = pd.concat([train, test],ignore_index=True)
y_df_all = pd.concat([train_y, test_y],ignore_index=True)

#X_df_all.head()

In [5]:
def getCodes():
    codes = []
    for comment, comment_id, line_no in zip(X["message"], X['comment_id'], X['line_number']):
        line_no = int(line_no)
        old_code = None

        old_folder = code_folder + comment_id + '/Old/'
        if(os.path.isdir(old_folder)):
            old_code = ""
            
            for filename in listdir(old_folder):
                code_lines = read_code_lines(old_folder, filename, line_no)
                lines = "\n".join(code_lines)
                old_code = old_code + lines
        else:
            print("no old")

        new_code = None
        new_folder = code_folder + comment_id + '/New/'
        if(os.path.isdir(new_folder)):
            new_code = ""
            for filename in listdir(new_folder):
                code_lines = read_code_lines(new_folder, filename, line_no)
                lines = "\n".join(code_lines)
                new_code = new_code + lines
        else:
            print("no new")

        codes.append((old_code, new_code))
    return codes

In [6]:
file_path = 'codes.pkl'
if os.path.exists(file_path):
    f = open(file_path, 'rb')
    codes = pickle.load(f)
else:
    codes = getCodes()
    f = open(file_path, 'wb')
    pickle.dump(codes, f)
    # sys.getsizeof(codes)
    len(pickle.dumps(codes))

In [7]:
'''X_df_all.drop(['Unnamed: 0', 'Unnamed: 0.1','Unnamed: 0.1.1', 'project', 'revision_id',
            'parent_revision','branch','file_name','reviewer_id','reviewer_name','reviewer_email',
            'author_id','Downloaded Patch.1', 'URL','change_id','request_id','line_number','patchset_number' ], axis=1, inplace=True)'''

X_df_all.drop(['Unnamed: 0', 'Unnamed: 0.1', 'project', 'revision_id',
            'parent_revision','branch','file_name','reviewer_id','reviewer_name','reviewer_email',
            'author_id','Downloaded Patch.1', 'URL','change_id','request_id','line_number','patchset_number' ], axis=1, inplace=True)
#X_df_all.columns,X_df_all.head()

In [8]:
code_df = pd.DataFrame(codes, columns=['old_code', 'new_code'])

In [9]:
X =pd.concat([X_df_all["message"], code_df["old_code"]], axis="columns")
#X.head()

In [10]:
y = pd.get_dummies(X_df_all['comment_group'])

In [11]:
def metrics(model, test_x, y_test, batch_size = 8):
    y_preds = model.predict(test_x, batch_size = batch_size)
    true_class = tf.argmax( y_test, 1 )
    predicted_class = tf.argmax( y_preds, 1 )

    # roc_auc = roc_auc_score(true_class, predicted_class, multi_class='ovr')
    # print(roc_auc)

    print(classification_report(true_class, predicted_class))
    cm = confusion_matrix(true_class, predicted_class)
    
    #ConfusionMatrixDisplay.from_predictions(true_class, predicted_class, display_labels=[str(l) for l in y_test.columns])
    return classification_report(true_class, predicted_class, output_dict=True), cm


In [12]:
X_3x = pd.concat([X, code_df["new_code"]], axis="columns")

In [13]:
def tokenize_text(tokenizer, sentence, total_len):
    tokens = tokenizer.encode_plus(sentence,max_length=total_len, truncation=True,add_special_tokens=True, padding="max_length")

    return tokens["input_ids"], tokens["attention_mask"]


In [14]:
total_token_slots = 512

comment_input_ids = []
comment_attention_masks = []
old_code_input_ids = []
old_code_attention_masks = []
new_code_input_ids = []
new_code_attention_masks = []
for cols, (comment, old_code, new_code) in X_3x.iterrows():
    comment = str(comment)
    old_code = str(old_code)
    new_code = str(new_code)

    #input_ids, attention_mask = tokenize_text(bert_tokenizer, comment, total_token_slots)
    input_ids, attention_mask = tokenize_text(codebert_tokenizer, comment, total_token_slots)
    comment_input_ids.append(input_ids)
    comment_attention_masks.append(attention_mask)

    input_ids, attention_mask = tokenize_text(codebert_tokenizer, old_code, total_token_slots)
    old_code_input_ids.append(input_ids)
    old_code_attention_masks.append(attention_mask)

    input_ids, attention_mask = tokenize_text(codebert_tokenizer, new_code, total_token_slots)
    new_code_input_ids.append(input_ids)
    new_code_attention_masks.append(attention_mask)

comment_input_ids           = np.array(comment_input_ids)
comment_attention_masks     = np.array(comment_attention_masks)
old_code_input_ids          = np.array(old_code_input_ids)
old_code_attention_masks    = np.array(old_code_attention_masks)
new_code_input_ids          = np.array(new_code_input_ids)
new_code_attention_masks    = np.array(new_code_attention_masks)



In [15]:
def read_metrics():
    metrics_df_path = None
    if use_function_metrics:
        metrics_df_path = 'data_new_metricsfunctionscope.csv'
    else:
        metrics_df_path = 'data_new_metrics.csv'
        
    metrics_df = pd.read_csv(metrics_df_path)

    print("metrics columns are eqaul check:", metrics_df['folderName'].equals(X_df_all['comment_id']))
    
    return metrics_df

In [16]:
metrics_df = read_metrics()
#metrics_df

metrics columns are eqaul check: True


In [17]:
def add_chosen_metrics(metrics_df, chosen_float_metric_columns):
    metrics_df = metrics_df.merge(X_df_all[chosen_float_metric_columns],left_index =True, right_index=True)
    chosen_float_metric_columns = metrics_df.columns 
    metrics_df = metrics_df.astype(np.float32)
    return metrics_df, chosen_float_metric_columns
def drop_error_columns(debugColumns):
    metrics_df.drop(debugColumns, axis=1, inplace=True)

In [18]:
chosen_float_metric_columns =['cyclomatic_complexity','comment_loc']

metrics_df["error"].replace(np.nan, "ok", inplace=True)

#dropIndices(metrics_df[metrics_df["error"] != "ok"].index)
#dropIndices(metrics_df.loc[metrics_df["isDupe"] != 0].index)
debugColumns = [
        "folderName",
        # "hasOldFile",#these are useful metrics don't drop
        # "hasNewFile",#these are useful metrics don't drop
        "numOldFiles",
        "numNewFiles",
        "error",
        "isDupe"
    ]

drop_error_columns(debugColumns)
metrics_df, chosen_metrics_columns = add_chosen_metrics(metrics_df, chosen_float_metric_columns)

In [19]:
print(chosen_metrics_columns)

Index(['anyInserted', 'anyDeleted', 'getMovedSrcs', 'UpdatedSrcs',
       'insertedIfConditions', 'deletedIfStmts', 'elseInserted', 'elseDeleted',
       'AnythingInLineMoved', 'AnythingInLineUpdated', 'AnythingInLineDeleted',
       'AnythingMovedIntoLine', 'AnythingInsertedIntoLine', 'EntireLineMoved',
       'EntireLineDeleted', 'stringsUpdated', 'magicStringsReplaced',
       'MovedBlocksInIfConditions', 'AddedOrUpdatedComments',
       'InsertedAssertConditions', 'InsertedTryCatch',
       'UpdatedValueAssignments', 'RemovedTryCatch', 'UpdatedFuncArguments',
       'hasOldFile', 'hasNewFile', 'cyclomatic_complexity', 'comment_loc'],
      dtype='object')


In [20]:
scaler = MinMaxScaler(clip = True)
scaler.fit(metrics_df)
metrics_df = scaler.transform(metrics_df)

In [21]:
def partition_input_dict(inputDict):
    returnDict = dict()
    returnDict["input_ids"] = inputDict["input_ids"][dataIndex]
    returnDict["token_type_ids"] = inputDict["token_type_ids"][dataIndex]
    returnDict["attention_mask"] = inputDict["attention_mask"][dataIndex]
    #instance_id =inputDict
    return returnDict

In [22]:
data_input = {"comment_input_ids": comment_input_ids ,
              "comment_attention_mask": comment_attention_masks,
                "code_input_ids": old_code_input_ids,
                "code_attention_mask": old_code_attention_masks,
             "metric" : metrics_df}

In [23]:
X_all = [comment_input_ids, comment_attention_masks,
            old_code_input_ids, old_code_attention_masks,
            metrics_df]

In [24]:
lstm_layer_1_name = 'lstm_layer_1'
lstm_layer_2_name = "lstm_layer_2"
bert_layer_name = 'tf_roberta_model'
def single_bert_layer_hidden_state(prefix):
    input_ids_dummy = tf.keras.layers.Input(shape=(total_token_slots,), name=(prefix+'input_ids'), dtype='int64')
    mask_dummy = tf.keras.layers.Input(shape=(total_token_slots,), name=(prefix+'attention_mask'), dtype='int64')
    codebert = TFAutoModel.from_pretrained("microsoft/codebert-base")
    # we access the transformer model within our bert object using the bert attribute (eg bert.bert instead of bert)
    # https://stackoverflow.com/questions/57000820/how-do-i-convert-bert-embeddings-into-a-tensor-for-feeding-into-an-lstm
    # https://github.com/huggingface/transformers/issues/7540#issuecomment-704155218
    # Huggingface Bert outputs a tuple, 
    # [0] => hidden state: (batch_size, seq_len, hidden_size), LSTM layer wants this
    # [1] => pooler output: basically just cls token through a tanh function
    embeddings_dummy = codebert(input_ids= input_ids_dummy, attention_mask=mask_dummy)[0]

    return input_ids_dummy,mask_dummy,codebert,embeddings_dummy


In [25]:
def get_bert_lstm_text_code():
    # two input layers, we ensure layer name variables match to dictionary keys in TF dataset
    comment_input_ids_dummy, comment_mask_dummy, comment_codebert, comment_embeddings_dummy = single_bert_layer_hidden_state("comment_")
    old_code_input_ids_dummy, old_code_mask_dummy, old_code_codebert, old_code_embeddings_dummy = single_bert_layer_hidden_state("code_")
    
    comment_x = tf.keras.layers.LSTM(50, dropout=0.3, recurrent_dropout=0.3, name=lstm_layer_1_name)(comment_embeddings_dummy)
    old_code_x = tf.keras.layers.LSTM(50, dropout=0.3, recurrent_dropout=0.3, name=lstm_layer_2_name)(old_code_embeddings_dummy)
    all_lstm = tf.keras.layers.Concatenate()([comment_x, old_code_x])
    
    # x = tf.keras.layers.Dense(1024, activation='relu')(embeddings_dummy)
    float_metrics_input_dummy = tf.keras.layers.Input(shape=(len(chosen_metrics_columns),), name='metric',dtype='float64')
    
    x_with_metrics = tf.keras.layers.Concatenate()([all_lstm, float_metrics_input_dummy])
    
    y = tf.keras.layers.Dense(4, activation='softmax', name='outputs')(x_with_metrics)

    # initialize model
    model = tf.keras.Model(inputs=[ comment_input_ids_dummy, comment_mask_dummy,
                                   old_code_input_ids_dummy, old_code_mask_dummy,
                                    float_metrics_input_dummy], outputs=y)

    # (optional) freeze bert layer
    #model.layers[2].trainable = False
    loss = tf.keras.losses.CategoricalCrossentropy()
    acc = tf.keras.metrics.CategoricalAccuracy()
    adam_optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)
    model.compile(loss=loss,optimizer=adam_optimizer,metrics=[acc])
    
    return model

In [26]:
def partition_input_dict(inputDict, dataIndex):
    returnDict = dict()
    returnDict["comment_input_ids"] = inputDict["comment_input_ids"][dataIndex]
    returnDict["comment_attention_mask"] = inputDict["comment_attention_mask"][dataIndex]
    returnDict["code_input_ids"] = inputDict["code_input_ids"][dataIndex]
    returnDict["code_attention_mask"] = inputDict["code_attention_mask"][dataIndex]
    returnDict["metric"] = inputDict["metric"][dataIndex]
    #instance_id =inputDict
    return returnDict


In [27]:
def generate_test_train_index( length, ratio):
    rand_array=np.arange(0,length,1)
    train, test =train_test_split(rand_array,test_size=ratio, shuffle=True )
    train=np.sort(train)
    test=np.sort(test)
    return train, test

In [28]:
y=y.values.tolist()
#print(y.shape)

In [29]:
y=np.array(y)

In [30]:
#five_fold = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED) 
from sklearn.model_selection import KFold

precision = [0.0, 0.0, 0.0, 0.0]
recall = [0.0, 0.0, 0.0, 0.0]
f_score = [0.0, 0.0, 0.0, 0.0]
accuracy = 0.0

rows, cols = (4, 4)
confusion = [[0.0]*cols]*rows

random_folding = KFold(n_splits=10, shuffle=True, random_state=SEED)
for train_index, test_index in random_folding.split(y):
    
    train_input_set = partition_input_dict(data_input, train_index)
    test_input = partition_input_dict(data_input, test_index)
    train_output = y[train_index,]
    test_output = y[test_index,]
    
    
    
    
    training_index, validation_index = generate_test_train_index(train_output.shape[0], 0.1)

    model_training_input = partition_input_dict(train_input_set, training_index)

    #print(len(model_training_input))

    validation_input = partition_input_dict(train_input_set, validation_index)
    model_training_output = train_output[training_index]
    validation_output = train_output[validation_index]
            
    
   # X_trval_fold, X_test_fold = X_all[train_index], X_all[test_index] 
   # y_trval_fold, y_test_fold = y[train_index], y[test_index]
    
    '''X_train_fold, X_val_fold, y_train_fold, y_val_fold = train_test_split(X_trval_fold, y_trval_fold,
                                   random_state=SEED, 
                                   test_size=0.11,
                                   shuffle=True)'''
    
    model = get_bert_lstm_text_code()
    #model.summary()
    callback = tf.keras.callbacks.EarlyStopping(
        monitor="val_loss",
        min_delta=0,
        patience=2,
        verbose=1,
        mode="min",
        baseline=None,
        restore_best_weights=True,)
    
    history = model.fit(model_training_input,
                        model_training_output, 
                        batch_size=4,
                        epochs=8,
                        validation_data=(validation_input, validation_output),
                        callbacks=[callback],
                        )
    res, confusion_mat = metrics(model, test_input, test_output)
    #print(res['1']['precision'])
    
    confusion += confusion_mat
    
    precision[0] += res['0']['precision']
    precision[1] += res['1']['precision']
    precision[2] += res['2']['precision']
    precision[3] += res['3']['precision']
    
    recall[0] += res['0']['recall']
    recall[1] += res['1']['recall']
    recall[2] += res['2']['recall']
    recall[3] += res['3']['recall']
    
    f_score[0] += res['0']['f1-score']
    f_score[1] += res['1']['f1-score']
    f_score[2] += res['2']['f1-score']
    f_score[3] += res['3']['f1-score']
    
    accuracy += res['accuracy']

All model checkpoint layers were used when initializing TFRobertaModel.

All the layers of TFRobertaModel were initialized from the model checkpoint at microsoft/codebert-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.
All model checkpoint layers were used when initializing TFRobertaModel.

All the layers of TFRobertaModel were initialized from the model checkpoint at microsoft/codebert-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Epoch 1/8
370/370 [==============================] - 520s 1s/step - loss: 1.2577 - categorical_accuracy: 0.4209 - val_loss: 1.2103 - val_categorical_accuracy: 0.4242
Epoch 2/8
370/370 [==============================] - 498s 1s/step - loss: 1.0871 - categorical_accuracy: 0.5493 - val_loss: 1.0030 - val_categorical_accuracy: 0.5879
Epoch 3/8
370/370 [==============================] - 500s 1s/step - loss: 0.8668 - categorical_accuracy: 0.6622 - val_loss: 0.9126 - val_categorical_accuracy: 0.6242
Epoch 4/8
370/370 [==============================] - 499s 1s/step - loss: 0.6347 - categorical_accuracy: 0.7682 - val_loss: 0.9508 - val_categorical_accuracy: 0.6303
Epoch 5/8
370/370 [==============================] - 500s 1s/step - loss: 0.4482 - categorical_accuracy: 0.8466 - val_loss: 1.0148 - val_categorical_accuracy: 0.6545
Epoch 5: early stopping


/home/amiangshu/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/amiangshu/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/amiangshu/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/amiangshu/anacon

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        21
           1       0.55      0.68      0.61        40
           2       0.75      0.62      0.68        72
           3       0.51      0.76      0.61        50

    accuracy                           0.60       183
   macro avg       0.45      0.52      0.48       183
weighted avg       0.56      0.60      0.57       183



All model checkpoint layers were used when initializing TFRobertaModel.

All the layers of TFRobertaModel were initialized from the model checkpoint at microsoft/codebert-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.
All model checkpoint layers were used when initializing TFRobertaModel.

All the layers of TFRobertaModel were initialized from the model checkpoint at microsoft/codebert-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


...

Epoch 1/8
370/370 [==============================] - 521s 1s/step - loss: 1.2058 - categorical_accuracy: 0.4473 - val_loss: 1.0674 - val_categorical_accuracy: 0.5636
Epoch 2/8
370/370 [==============================] - 504s 1s/step - loss: 1.0165 - categorical_accuracy: 0.5777 - val_loss: 1.0199 - val_categorical_accuracy: 0.5515
Epoch 3/8
370/370 [==============================] - 504s 1s/step - loss: 0.8163 - categorical_accuracy: 0.6723 - val_loss: 1.0803 - val_categorical_accuracy: 0.5515
Epoch 4/8
370/370 [==============================] - 503s 1s/step - loss: 0.5932 - categorical_accuracy: 0.7858 - val_loss: 1.0956 - val_categorical_accuracy: 0.5515
Epoch 4: early stopping
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        22
           1       0.59      0.57      0.58        47
           2       0.53      0.83      0.65        69
           3       0.55      0.36      0.43        45

    accuracy                           0

All model checkpoint layers were used when initializing TFRobertaModel.

All the layers of TFRobertaModel were initialized from the model checkpoint at microsoft/codebert-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.
All model checkpoint layers were used when initializing TFRobertaModel.

All the layers of TFRobertaModel were initialized from the model checkpoint at microsoft/codebert-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Epoch 1/8
371/371 [==============================] - 724s 2s/step - loss: 1.2186 - categorical_accuracy: 0.4517 - val_loss: 1.0798 - val_categorical_accuracy: 0.5515
Epoch 2/8
371/371 [==============================] - 701s 2s/step - loss: 0.9831 - categorical_accuracy: 0.6050 - val_loss: 1.1107 - val_categorical_accuracy: 0.5515
Epoch 3/8
371/371 [==============================] - 693s 2s/step - loss: 0.7978 - categorical_accuracy: 0.6941 - val_loss: 1.0514 - val_categorical_accuracy: 0.5818
Epoch 4/8
371/371 [==============================] - 696s 2s/step - loss: 0.5904 - categorical_accuracy: 0.7927 - val_loss: 1.1359 - val_categorical_accuracy: 0.6182
Epoch 5/8
371/371 [==============================] - 697s 2s/step - loss: 0.4010 - categorical_accuracy: 0.8805 - val_loss: 1.2384 - val_categorical_accuracy: 0.5879
Epoch 5: early stopping
              precision    recall  f1-score   support

           0       0.50      0.18      0.26        17
           1       0.60      0.68    

All model checkpoint layers were used when initializing TFRobertaModel.

All the layers of TFRobertaModel were initialized from the model checkpoint at microsoft/codebert-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.
All model checkpoint layers were used when initializing TFRobertaModel.

All the layers of TFRobertaModel were initialized from the model checkpoint at microsoft/codebert-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Epoch 1/8
371/371 [==============================] - 727s 2s/step - loss: 1.2328 - categorical_accuracy: 0.4396 - val_loss: 1.1653 - val_categorical_accuracy: 0.5030
Epoch 2/8
371/371 [==============================] - 701s 2s/step - loss: 1.0509 - categorical_accuracy: 0.5820 - val_loss: 1.0996 - val_categorical_accuracy: 0.5152
Epoch 3/8
371/371 [==============================] - 701s 2s/step - loss: 0.8563 - categorical_accuracy: 0.6793 - val_loss: 1.1023 - val_categorical_accuracy: 0.5273
Epoch 4/8
371/371 [==============================] - 706s 2s/step - loss: 0.6839 - categorical_accuracy: 0.7461 - val_loss: 1.0968 - val_categorical_accuracy: 0.6061
Epoch 5/8
371/371 [==============================] - 705s 2s/step - loss: 0.5217 - categorical_accuracy: 0.8244 - val_loss: 1.1167 - val_categorical_accuracy: 0.6182
Epoch 6/8
371/371 [==============================] - 701s 2s/step - loss: 0.3660 - categorical_accuracy: 0.8805 - val_loss: 1.2888 - val_categorical_accuracy: 0.5697
Epoc

In [31]:
print(precision)
print(recall)
print(f_score)
print(accuracy)

[3.6666666666666665, 5.994795674379037, 7.21297815585316, 5.4290836979287365]
[1.566305753070459, 6.606567844638649, 7.426028947211191, 5.638014782375247]
[2.131085603049528, 6.252236032006898, 7.236260503584873, 5.437528735682273]
6.290938569627094


In [32]:
# final performance values
print([x/10 for x in precision])
print([x/10 for x in recall])
print([x/10 for x in f_score])
print(accuracy/10)
print([[item / 10 for item in subl] for subl in confusion])

[0.36666666666666664, 0.5994795674379036, 0.721297815585316, 0.5429083697928736]
[0.15663057530704588, 0.6606567844638649, 0.7426028947211191, 0.5638014782375247]
[0.21310856030495282, 0.6252236032006898, 0.7236260503584873, 0.5437528735682273]
0.6290938569627094
[[2.3, 4.1, 4.6, 4.8], [0.7, 29.5, 8.3, 6.0], [0.4, 9.3, 58.3, 10.4], [1.1, 6.4, 11.7, 24.9]]
